# Creating Animated Figures
This file creates an animated figure (GIF file) of the specified video and experimental electromyography (EMG) data collected.

## Contributors
Alex Duman, Ph.D. - primary and sole author of this script.

In [42]:
# Importing necessary packages
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import glob
import imageio

## Input Arguments
Adjust these values in order to create a new GIF with different trials or data.

<b>fileName</b> - the name of the file including the path where it is located

<b>movieName</b> - the name of the movie file of interest, also including path as well

<b>frameInc</b> - the increment to sample movie frames at, e.g. setting equal to 1 means the GIF will cycle through each                     frame of the movie while setting equal to 2 will only show every second frame of the movie in the final                     GIF

<b>duration</b> - length in seconds of data you want shown in the GIF (e.g. 10 will produce a GIF of the first 10 seconds                     of data), must be equal or less than the shorter of the data and movie file

In [43]:
# INPUTS
fileName = 'Run_Forced_Rotation_EMG.csv'
movieName = 'Run_Forced_Rotation.mp4'
frameInc = 2
duration = 4 # seconds
trialTitle = 'Forced Rotation'

In [44]:
# Loading in Data
Data = pd.read_csv(os.getcwd() + os.path.sep + 'data' + os.path.sep + fileName)

# Want to make sure Data starts at time zero
if Data.time[0] != 0:
    Data.time = Data.time - Data.time[0] # adjust data so that initial time point is at time zero
D_hz = 1/(Data.time[1] - Data.time[0]) # want to know data's sampling frequency to sync up with movie

In [45]:
# Loading in Movie
cap = cv2.VideoCapture(os.getcwd() + os.path.sep + 'data' + os.path.sep + movieName) # here cap(ture) represents the movieObj
nFrames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
M_hz = cap.get(cv2.CAP_PROP_FPS) # frame rate of movie in fps or Hz
Tot_frames = cap.get(cv2.CAP_PROP_FRAME_COUNT)

# Determine Sample Ratio to synchronize video and data
sampleRatio = D_hz/M_hz

In [46]:
# Determine y-axes bounds for data
def Find_y_bounds(y):
    '''This function determines the minimum and maximum values for the vector y
    so that when it is plotted it displays [y_min, y_max] where y_min and
    y_max are 5% beyond the actualy minima and maxima values of y.
    
    INPUTS
    y - numerical vector containing data interested in visualizing
    
    OUTPUTS
    y_bound - the y-axis bounds for y that give a 5% buffer on either end
              [y_min_bound, y_max_bound]
    '''
    delta_y = max(y)-min(y)
    y_max_bound = max(y)+0.05*delta_y
    y_min_bound = min(y)-0.05*delta_y
    y_bound = np.array([y_min_bound, y_max_bound])
    return y_bound

# Finding each y-axes bounds for data
y_bounds = np.zeros((len(Data.columns),2))
for i in range(1,(len(Data.columns))):
    y_bounds[i,:] = Find_y_bounds(Data.iloc[:,i]) # reported as [min, max]

In [47]:
%%time
# Creating Series of Figures and Saving them as Images (to be later compiled into a single GIF)
gif_frame = 1 # start off with frame 1 of gif

for m in range(0, round(duration*M_hz), frameInc): # looping through all the frames of interest
    
    data_idx = round(m*sampleRatio) # index for data corresponding to specific movie frame
    
    if data_idx <= len(Data.iloc[:,0]): # only want to create figure to add to gif if we have data

        # Creating Figure
        fig = plt.figure(figsize = [13.33, 7.5], dpi = 300.0)

        y_labels = ['LG Activity', 'MG Activity', 'RF Activity']

        for i in range(1,5): # loop through to create each subplot
            plt.subplot(4,1,i)
    
            if i == 1:
                # Finding Movie Frame of Interest
                cap.set(cv2.CAP_PROP_POS_FRAMES, m)
                rval, frame_BGR = cap.read() # reads in Blue-Green-Red
                frame = cv2.cvtColor(frame_BGR, cv2.COLOR_BGR2RGB) # formatted in RGB for matplotlib plotting
                
                # Movie axis
                plt.axis('off')
                plt.axes(arg = [0, 0.70, 1, 0.3]) # [left, bottom, width, height] all in fractional terms
                plt.imshow(frame)
                plt.text(30, 80, trialTitle, color = 'w', fontsize = 'large')
                plt.axis('off')
            else:
                # Data axes
                plt.ylabel(y_labels[i-2])
                if i == 4:
                    plt.xlabel('Time (s)')
                plt.plot(Data.time, Data.iloc[:,i-1], color = [1,0.6,0.6])
                plt.plot(Data.time, Data.iloc[:,i+2], color = [0.6,0.6,1])
                plt.plot(Data.time[0:data_idx], Data.iloc[0:data_idx,i-1], color = [1,0,0])
                plt.plot(Data.time[0:data_idx], Data.iloc[0:data_idx,i+2], color = [0,0,1])
                plt.xlim((0, duration))
                plt.ylim((min([y_bounds[i-1,0], y_bounds[i+2,0]]), max([y_bounds[i-1,1], y_bounds[i+2,1]])))

        # Creating Appropriate Name to Save Figure (single frame within GIF)
        if gif_frame < 10:
            gif_frame_num = '000' + str(gif_frame)
        elif gif_frame < 100:
            gif_frame_num = '00' + str(gif_frame)
        elif gif_frame < 1000:
            gif_frame_num = '0' + str(gif_frame)
        else: # only allows for gifs of <10,000 frames (anything that long should be considered for a video)
            gif_frame_num = str(gif_frame)

        # Save figure as image and then close it to reduce memory usage
        plt.savefig('GIF_frame_' + gif_frame_num + '.jpg')
        plt.close(fig)
        
        # Iterate to next frame in gif
        gif_frame = gif_frame + 1

Wall time: 2min 20s


In [48]:
%%time
# Creating GIF from Image Files
filenames = glob.glob(os.getcwd() + os.path.sep + 'GIF_frame_*.jpg')
with imageio.get_writer(os.getcwd() + os.path.sep + 'results' + os.path.sep + movieName[:-4] + '.gif', mode = 'I') as writer:
    for file in filenames:
        image = imageio.imread(file)
        writer.append_data(image)

# Removing GIF Images
for file in filenames:
    os.remove(file)

Wall time: 2min 15s


In [49]:
print('All finished exporting GIF!')

All finished exporting GIF!
